In [2]:
import numpy as np
x_train=np.load('../../../datasets/ORL/d_orl.npy')
x_train=x_train.reshape(x_train.shape[0]*x_train.shape[1],400)
y=np.load('../../../datasets/ORL/l_orl.npy')
y=y.reshape(y.shape[0]*y.shape[1],)
alpha_trans1=np.load('../../../datasets/ORL/Sb_orl.npy',allow_pickle=True)
alpha_trans1=np.asarray(alpha_trans1).astype('float32') 

In [3]:
import glob
import numpy as np
import random
import tensorflow.compat.v1 as tf
import os

batchSize=400
from sklearn.metrics.pairwise import euclidean_distances
from scipy.linalg import fractional_matrix_power
from scipy import linalg
import sklearn
from sklearn.cluster import KMeans
from tensorflow.keras import backend as B
k_cluster = 40
from tensorflow.keras.layers import Input,Dense,Lambda,Layer
from tensorflow.keras.models import Model
import sys
import numpy as np
from scipy.io import loadmat
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from munkres import Munkres
import cv2
from numpy import save


def make_cost_matrix(c1, c2):
    uc1 = np.unique(c1)
    uc2 = np.unique(c2)
    l1 = uc1.size
    l2 = uc2.size
    for a in range (1):
        if uc1.shape==uc2.shape:
            m = np.ones([l1, l2])
            for i in range(l1):
                it_i = np.nonzero(c1 == uc1[i])[0]
                for j in range(l2):
                    it_j = np.nonzero(c2 == uc2[j])[0]
                    m_ij = np.intersect1d(it_j, it_i)
                    m[i,j] =  -m_ij.size
        else:
            print('assertion handeled')
            return
            #break
    return m

def translate_clustering(clt, mapper):
    return np.array([ mapper[i] for i in clt ])

def accuracy(cm):
    return np.trace(cm, dtype=float) / np.sum(cm)

def rectify_label(labels, classes):
    num_labels = len(np.unique(classes))
    cm = confusion_matrix(classes, labels, labels=range(num_labels)) # gets the confusion matrix
    cost_matrix = make_cost_matrix(labels, classes)
    if cost_matrix is None:
        return (None,None)
    else:
        m = Munkres()
        indexes = m.compute(cost_matrix)
        mapper = { old: new for (old, new) in indexes }
        new_labels = translate_clustering(labels, mapper)
        new_cm = confusion_matrix(classes, new_labels, labels=range(num_labels))
        return new_labels,accuracy(new_cm)

def arch():
    encoding_dim = 390  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats
    input_img = Input(shape=(400,))
    encoded_layer_2 = Dense(397, activation='tanh')(input_img)
    encoded_layer_3 = Dense(395, activation='tanh')(encoded_layer_2)
    encoded_layer_4 = Dense(393, activation='tanh')(encoded_layer_3)
    encoded = Dense(encoding_dim, activation='tanh')(encoded_layer_4)
    decoded_layer_2 = Dense(393, activation='tanh')(encoded)
    decoded_layer_3 = Dense(395, activation='tanh')(decoded_layer_2)
    decoded_layer_4 = Dense(397, activation='tanh')(decoded_layer_3)
    decoded = Dense(400, activation='tanh')(decoded_layer_4)
    encoder=Model(input_img,encoded)
    autoencoder=Model(input_img,decoded)
    return encoder,encoding_dim,autoencoder



def assign_func(z_test,z_ic):
    dst = np.array(np.sum(((z_test-z_ic[0]) **2),axis=1))
    for index in range(1, z_ic.shape[0]):
        col = np.sum(((z_test- z_ic[index])**2), axis=1)
        dst = np.vstack((dst, col))
    dst = dst.T      
    
    mean_dst = np.mean(dst, axis=1)
    q = np.maximum(0.0, np.tile(mean_dst, (dst.shape[1], 1)).T - dst)
    #print(q.shape)
    num_centers = q.shape[1]
    weight = 1.0 / (q.sum(axis=0) + 1e-9)
    weight *= num_centers / weight.sum()
    q = (q ** 2.0) * weight
    q = (q.T / (q.sum(axis=1) + B.epsilon())).T
    return q

def asl():
    def myloss(y_true, y_pred):
        l1norm= l1norm_func(y_pred,y_true)
        l2norm= l2norm_func(y_pred,y_true)
        orthonormal= orthonormality_func(y_pred)
        structpreserve=structurePreserve_func(y_pred,alpha_trans1[g][:,:])
        return orthonormal+l1norm+l2norm+structpreserve
    return myloss

def l1norm_func(z,nt):
    q1=0.002*(tf.reduce_sum(tf.abs(tf.subtract(z,nt)),axis=1))
    return q1

def l2norm_func(z,nt):
    q2=0.02*(tf.sqrt(tf.reduce_sum(tf.square(z-nt),axis=1)))
    return q2

def structurePreserve_func(z,ta):
    tz=tf.transpose(z)
    tz=tf.reshape(tz,(encoding_dim,batchSize))
    taa=tf.transpose(ta)
    taz=tf.matmul(tz,taa)
    taz=tf.reshape(taz,(encoding_dim,batchSize))
    q4=0.02*(tf.sqrt(tf.reduce_sum(tf.square(tz-taz))))
    q4=tf.transpose(q4)
    return q4


def orthonormality_func(z):
    z=[(z[i,:]/(tf.norm(z[i,:],2)))for i in range (batchSize)]
    transz=tf.transpose(z)
    q3=tf.tensordot(transz,z,axes=1)
    I=tf.eye(encoding_dim)
    q5=0.002*(tf.sqrt(tf.reduce_sum(tf.square(q3-I))))
    return q5



In [4]:

encoder,encoding_dim,autoencoder=arch()
print('loading pre trained model')
encoder.load_weights('../../../checkpoints/chk_orl/sdsc_orl.ckpt')
z_test=encoder.predict(x_train)
ic= np.load('../../../checkpoints/chk_orl/sdsc_icorl.npy')
assignment = assign_func(z_test, ic)
predictedlabels = np.argmax(assignment, axis=1)
rectified_label, acc=rectify_label(predictedlabels, y)
nmi=sklearn.metrics.normalized_mutual_info_score(y, rectified_label)
f1=sklearn.metrics.f1_score(y, rectified_label,average='micro')
prec=sklearn.metrics.precision_score(y, rectified_label,average='weighted')
print(acc,nmi,f1,prec)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
loading pre trained model
0.9075 0.9466350858615871 0.9075 0.9261012598512598
